# Tracing with Traceable

LangSmith makes it easy to trace any LLM application, no LangChain required!

#### What is a Trace actually?

 Each trace is made of 1 or more "runs" representing key event spans in your app. Each run is a structured log with a name, run_type, inputs / outputs, start/end times, as well as tags and other metadata. Runs within a trace are nested, forming a hierarchy referred to as a "run tree."

## Setup

Make sure you set your environment variables, including your OpenAI API key.

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = ""
os.environ["LANGCHAIN_API_KEY"] = ""
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "langsmith-academy"

In [ ]:
from dotenv import load_dotenv

# I have my env variables defined in a .env file
load_dotenv(dotenv_path="../../.env", override=True)

#### Set up our VectorDB retriever

In [ ]:
import nest_asyncio
from utils import get_vector_db_retriever

nest_asyncio.apply()
retriever = get_vector_db_retriever()

## Tracing a RAG app with @traceable

![Simple RAG](../../images/simple_rag.png)

#### Using the @traceable decorator



The @traceable decorator handles the RunTree lifecycle for you, and allows you to add tags and metadata to your runs.

In [12]:
from langsmith import traceable
from openai import OpenAI
from typing import List

openai_client = OpenAI()

@traceable(run_type="retriever")
def retrieve_documents(question: str):
    return retriever.invoke(question)

@traceable(run_type="chain")
def generate_response(question: str, documents):
    formatted_docs = "\n\n".join(doc.page_content for doc in documents)
    rag_system_prompt = """You are an assistant for question-answering tasks. 
    Use the following pieces of retrieved context to answer the latest question in the conversation. 
    If you don't know the answer, just say that you don't know. 
    Use three sentences maximum and keep the answer concise.
    """
    messages = [
        {
            "role": "system",
            "content": rag_system_prompt
        },
        {
            "role": "user",
            "content": f"Context: {formatted_docs} \n\n Question: {question}"
        }
    ]
    return call_openai(messages)

@traceable(run_type="llm")
def call_openai(
    messages: List[dict], model: str = "gpt-4o-mini", temperature: float = 0.0
) -> str:
    return openai_client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )

@traceable(run_type="chain")
def langsmith_rag(question: str):
    documents = retrieve_documents(question)
    response = generate_response(question, documents)
    return response.choices[0].message.content


In [ ]:
question = "How can I trace with the @traceable decorator?"
ai_answer = langsmith_rag(question)
print(ai_answer)

#### Let's take a look in LangSmith!